In [2]:
from sqlalchemy import *
import pandas as pd
engine = create_engine('sqlite:///sqlite_db_pythonsqlite.db')
table_names = engine.table_names()
print(table_names)

['Bookings', 'Facilities', 'Members']


/var/folders/qv/d4946vpn3qjf14wqxbxdnq5m0000gn/T/ipykernel_1601/4197245412.py:4: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  table_names = engine.table_names()


In [3]:
with engine.connect() as con:
    r = con.execute('SELECT * FROM Bookings')
    B_df = pd.DataFrame(r.fetchall())
    B_df.columns = r.keys()
print(B_df.head())

   bookid  facid  memid            starttime  slots
0       0      3      1  2012-07-03 11:00:00      2
1       1      4      1  2012-07-03 08:00:00      2
2       2      6      0  2012-07-03 18:00:00      2
3       3      7      1  2012-07-03 19:00:00      2
4       4      8      1  2012-07-03 10:00:00      1


In [4]:
F_df = pd.read_sql_query('SELECT * FROM Facilities', engine)
print(F_df.head())

   facid             name  membercost  guestcost  initialoutlay  \
0      0   Tennis Court 1         5.0       25.0          10000   
1      1   Tennis Court 2         5.0       25.0           8000   
2      2  Badminton Court         0.0       15.5           4000   
3      3     Table Tennis         0.0        5.0            320   
4      4   Massage Room 1         9.9       80.0           4000   

   monthlymaintenance  
0                 200  
1                 200  
2                  50  
3                  10  
4                3000  


In [5]:
M_df = pd.read_sql_query('SELECT * FROM Members', engine)
print(M_df.head())

   memid   surname firstname                       address  zipcode  \
0      0     GUEST     GUEST                         GUEST        0   
1      1     Smith    Darren    8 Bloomsbury Close, Boston     4321   
2      2     Smith     Tracy  8 Bloomsbury Close, New York     4321   
3      3    Rownam       Tim        23 Highway Way, Boston    23423   
4      4  Joplette    Janice    20 Crossing Road, New York      234   

        telephone recommendedby             joindate  
0  (000) 000-0000                2012-07-01 00:00:00  
1    555-555-5555                2012-07-02 12:02:05  
2    555-555-5555                2012-07-02 12:08:23  
3  (844) 693-0723                2012-07-03 09:32:15  
4  (833) 942-4710             1  2012-07-03 10:25:05  


In [21]:
Q_8 = pd.read_sql_query("""SELECT b.starttime, f.name, m.firstname, f.membercost * b.slots AS cost
FROM Bookings AS b
JOIN Facilities AS f
JOIN Members AS m
WHERE b.starttime LIKE '2012-09-14%' AND f.membercost * b.slots > 30.0
ORDER BY cost DESC;""", engine)
print(Q_8)

               starttime            name  firstname  cost
0    2012-09-14 08:00:00  Massage Room 1      GUEST  59.4
1    2012-09-14 08:00:00  Massage Room 1     Darren  59.4
2    2012-09-14 08:00:00  Massage Room 1      Tracy  59.4
3    2012-09-14 08:00:00  Massage Room 1        Tim  59.4
4    2012-09-14 08:00:00  Massage Room 1     Janice  59.4
..                   ...             ...        ...   ...
367  2012-09-14 09:30:00  Massage Room 2  Millicent  39.6
368  2012-09-14 09:30:00  Massage Room 2   Hyacinth  39.6
369  2012-09-14 09:30:00  Massage Room 2       John  39.6
370  2012-09-14 09:30:00  Massage Room 2      Erica  39.6
371  2012-09-14 09:30:00  Massage Room 2     Darren  39.6

[372 rows x 4 columns]


In [37]:
Q9 = pd.read_sql_query("""SELECT name, firstname, surname, 
CASE WHEN memid = 0
THEN guestcost * slots
ELSE membercost * slots
END AS cost, starttime
FROM (SELECT * FROM Bookings AS b
JOIN Facilities AS f ON b.facid = f.facid
JOIN Members AS m ON b.memid = m.memid
WHERE starttime LIKE '2012-09-14%' 
AND CASE WHEN m.memid = 0 THEN guestcost * slots
ELSE membercost * slots END > 30)
ORDER BY cost DESC;""", engine)
print(Q9)

              name firstname  surname   cost            starttime
0   Massage Room 2     GUEST    GUEST  320.0  2012-09-14 11:00:00
1   Massage Room 1     GUEST    GUEST  160.0  2012-09-14 09:00:00
2   Massage Room 1     GUEST    GUEST  160.0  2012-09-14 13:00:00
3   Massage Room 1     GUEST    GUEST  160.0  2012-09-14 16:00:00
4   Tennis Court 2     GUEST    GUEST  150.0  2012-09-14 17:00:00
5   Tennis Court 1     GUEST    GUEST   75.0  2012-09-14 16:00:00
6   Tennis Court 1     GUEST    GUEST   75.0  2012-09-14 19:00:00
7   Tennis Court 2     GUEST    GUEST   75.0  2012-09-14 14:00:00
8     Squash Court     GUEST    GUEST   70.0  2012-09-14 09:30:00
9   Massage Room 1    Jemima  Farrell   39.6  2012-09-14 14:00:00
10    Squash Court     GUEST    GUEST   35.0  2012-09-14 12:30:00
11    Squash Court     GUEST    GUEST   35.0  2012-09-14 15:00:00


In [88]:
Q10 = pd.read_sql_query("""WITH rev AS (SELECT name, Facilities.facid AS id, 
SUM(CASE WHEN memid = 0 THEN guestcost * slots ELSE membercost * slots END) AS revenue 
FROM Facilities
JOIN Bookings ON Facilities.facid = Bookings.facid
GROUP BY name)
SELECT f.name, rev.revenue 
FROM Facilities AS f
JOIN rev ON f.facid = rev.id
WHERE rev.revenue < 1000
ORDER BY rev.revenue""", engine)
print(Q10)

            name  revenue
0   Table Tennis      180
1  Snooker Table      240
2     Pool Table      270


In [113]:
Q11 = pd.read_sql_query("""SELECT m1.firstname, m1.surname, m1.recommendedby AS rec, m2.firstname AS recfirstname, 
m2.surname AS recsurname
FROM Members AS m1
JOIN Members AS m2 ON rec = m2.memid
WHERE rec > 0 
ORDER BY m1.firstname, m1.surname""", engine)
print(Q11)

    firstname            surname rec recfirstname recsurname
0        Anna          Mackenzie   1       Darren      Smith
1        Anne              Baker   9       Ponder   Stibbons
2     Charles               Owen   1       Darren      Smith
3       David              Jones   4       Janice   Joplette
4       David             Pinker  13       Jemima    Farrell
5     Douglas              Jones  11        David      Jones
6       Erica            Crumpet   2        Tracy      Smith
7    Florence              Bader   9       Ponder   Stibbons
8      Gerald            Butters   1       Darren      Smith
9   Henrietta             Rumney  20      Matthew    Genting
10      Henry  Worthington-Smyth   2        Tracy      Smith
11       Jack              Smith   1       Darren      Smith
12     Janice           Joplette   1       Darren      Smith
13       Joan             Coplin  16      Timothy      Baker
14       John               Hunt  30    Millicent    Purview
15    Matthew           

In [123]:
Q12 = pd.read_sql_query("""SELECT f.name, b.memid, m.firstname, m.surname
FROM Bookings AS b
JOIN Facilities AS f ON f.facid = b.facid
JOIN Members AS m ON m.memid = b.memid
WHERE b.memid != 0""", engine)
print(Q12)

                name  slots  memid firstname            surname
0       Table Tennis      2      1    Darren              Smith
1     Massage Room 1      2      1    Darren              Smith
2      Snooker Table      2      1    Darren              Smith
3         Pool Table      1      1    Darren              Smith
4         Pool Table      1      1    Darren              Smith
...              ...    ...    ...       ...                ...
3155      Pool Table      2     29     Henry  Worthington-Smyth
3156      Pool Table      1     29     Henry  Worthington-Smyth
3157      Pool Table      1     21      Anna          Mackenzie
3158      Pool Table      1     16   Timothy              Baker
3159      Pool Table      1     29     Henry  Worthington-Smyth

[3160 rows x 5 columns]


In [140]:
Q13 = pd.read_sql_query("""SELECT f.name AS facility, slots AS usage, strftime('%m', starttime) AS month, 
m.firstname, m.surname 
FROM Bookings AS b
JOIN Facilities AS f ON f.facid = b.facid
JOIN Members AS m ON m.memid = b.memid
WHERE b.memid != 0""", engine)
print(Q13)

            facility  usage month firstname            surname
0       Table Tennis      2    07    Darren              Smith
1     Massage Room 1      2    07    Darren              Smith
2      Snooker Table      2    07    Darren              Smith
3         Pool Table      1    07    Darren              Smith
4         Pool Table      1    07    Darren              Smith
...              ...    ...   ...       ...                ...
3155      Pool Table      2    09     Henry  Worthington-Smyth
3156      Pool Table      1    09     Henry  Worthington-Smyth
3157      Pool Table      1    09      Anna          Mackenzie
3158      Pool Table      1    09   Timothy              Baker
3159      Pool Table      1    09     Henry  Worthington-Smyth

[3160 rows x 5 columns]
